In [1]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Input, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import accuracy_score


In [2]:
# Import the data
batchsize = 8
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2
)

In [3]:
train_data = train_datagen.flow_from_directory(
    r'C:\Users\INFINIX\Driver Drowsiness Detection System\Prepared_Data\train',
    target_size=(224, 224),  # MobileNetV2 model input shape
    batch_size=batchsize,
    class_mode='categorical',
    subset='training'
)

validation_data = train_datagen.flow_from_directory(
    r'C:\Users\INFINIX\Driver Drowsiness Detection System\Prepared_Data\train',
    target_size=(224, 224),  # MobileNetV2 model input shape
    batch_size=batchsize,
    class_mode='categorical',
    subset='validation'
)


Found 64105 images belonging to 2 classes.
Found 16025 images belonging to 2 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(
    r'C:\Users\INFINIX\Driver Drowsiness Detection System\Prepared_Data\test',
    target_size=(224, 224),  # MobileNetV2 model input shape
    batch_size=batchsize,
    class_mode='categorical'
)

Found 4768 images belonging to 2 classes.


In [5]:
# Model training
bmodel = MobileNetV2(include_top=False, weights='imagenet', input_tensor=Input(shape=(224, 224, 3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2, activation='softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs=hmodel)

for layer in bmodel.layers:
    layer.trainable = False



9406464/9406464 [==============================] - 12s 1us/step


In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[0][0]']        

In [7]:
checkpoint = ModelCheckpoint(
    r'C:\Users\INFINIX\Driver Drowsiness Detection System\models\model.h5',
    monitor='val_loss',
    save_best_only=True,
    verbose=3
)

earlystop = EarlyStopping(
    monitor='val_loss',
    patience=7,
    verbose=3,
    restore_best_weights=True
)

learning_rate = ReduceLROnPlateau(
    monitor='val_loss',
    patience=3,
    verbose=3,
)

callbacks = [checkpoint, earlystop, learning_rate]

In [8]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit_generator(
    train_data,
    steps_per_epoch=train_data.samples//batchsize,
    validation_data=validation_data,
    validation_steps=validation_data.samples//batchsize,
    callbacks=callbacks,
    epochs=5
)


C:\Users\INFINIX\AppData\Local\Temp\ipykernel_12936\2642282653.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/5


8013/8013 [==============================] - ETA: 0s - loss: 0.3507 - accuracy: 0.8716
Epoch 1: val_loss improved from inf to 0.32002, saving model to C:\Users\INFINIX\Driver Drowsiness Detection System\models\model.h5


C:\Users\INFINIX\.conda\envs\yogesh\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8013/8013 [==============================] - 2649s 330ms/step - loss: 0.3507 - accuracy: 0.8716 - val_loss: 0.3200 - val_accuracy: 0.8973 - lr: 0.0010
Epoch 2/5
8013/8013 [==============================] - ETA: 0s - loss: 0.2432 - accuracy: 0.8996
Epoch 2: val_loss improved from 0.32002 to 0.27523, saving model to C:\Users\INFINIX\Driver Drowsiness Detection System\models\model.h5
8013/8013 [==============================] - 2617s 327ms/step - loss: 0.2432 - accuracy: 0.8996 - val_loss: 0.2752 - val_accuracy: 0.8914 - lr: 0.0010
Epoch 3/5
8013/8013 [==============================] - ETA: 0s - loss: 0.2344 - accuracy: 0.9060
Epoch 3: val_loss did not improve from 0.27523
8013/8013 [==============================] - 2575s 321ms/step - loss: 0.2344 - accuracy: 0.9060 - val_loss: 0.3191 - val_accuracy: 0.8945 - lr: 0.0010
Epoch 4/5
6009/8013 [=====================>........] - ETA: 8:50 - loss: 0.2256 - accuracy: 0.9126

KeyboardInterrupt: 

In [10]:
# Evaluate the model
acc_tr, loss_tr = model.evaluate(train_data)
print("Training Accuracy:", acc_tr)
print("Training Loss:", loss_tr)

8014/8014 [==============================] - 1945s 243ms/step - loss: 0.1123 - accuracy: 0.9559
Training Accuracy: 0.11230576038360596
Training Loss: 0.9558692574501038


In [11]:

acc_vr, loss_vr = model.evaluate(validation_data)
print("Validation Accuracy:", acc_vr)
print("Validation Loss:", loss_vr)

2004/2004 [==============================] - 489s 244ms/step - loss: 0.2881 - accuracy: 0.9032
Validation Accuracy: 0.28807783126831055
Validation Loss: 0.9032137393951416


In [12]:
acc_test, loss_test = model.evaluate(test_data)
print("Test Accuracy:", acc_test)
print("Test Loss:", loss_test)


596/596 [==============================] - 83s 139ms/step - loss: 0.4032 - accuracy: 0.8664
Test Accuracy: 0.4031761884689331
Test Loss: 0.8664010167121887
